# Digit Recognition using Paint and SVM SVR Classifier

In [1]:
import pyscreenshot as ss
import time

In [2]:
imgdataset="img-dataset/"
print('**************************')
for i in range(0,10):
    for j in range(1,5):
        print("Draw----",i," (",j,")")
        time.sleep(8)
        img=ss.grab(bbox=(25,231,305,511))
        print("Saved----")
        img.save(imgdataset+str(i)+'/'+str(j)+'.png')
    time.sleep(2)
    print('**************************')

**************************
Draw---- 0  ( 1 )
Saved----
Draw---- 0  ( 2 )
Saved----
Draw---- 0  ( 3 )
Saved----
Draw---- 0  ( 4 )
Saved----
**************************
Draw---- 1  ( 1 )
Saved----
Draw---- 1  ( 2 )


In [ ]:
import cv2
import csv
import glob

In [ ]:
header=['label']
for i in range(0,784):
    header.append('pixel'+str(i))
with open('dataset.csv','a') as f:
    writer=csv.writer(f)
    writer.writerow(header)
for label in range(10):
    dirList=glob.glob('img-dataset/'+str(label)+'/*.png')
    for img_path in dirList:
        im=cv2.imread(img_path) #path for image file
        im_gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        im_gray=cv2.GaussianBlur(im_gray,(15,15),0)
        roi=cv2.resize(im_gray,(28,28),interpolation=cv2.INTER_AREA)
        data=[]
        data.append(label)
        rows,cols=roi.shape

        for i in range(rows):
            for j in range(cols):
                k=roi[i,j]
                if k>100:
                    k=1
                else:
                    k=0
                data.append(k)
        with open('dataset.csv','a') as f:
            writer=csv.writer(f)
            writer.writerow(data)

In [ ]:
import pandas as pd
from sklearn.utils import shuffle
data=pd.read_csv('dataset.csv')
data=shuffle(data)
data

In [ ]:
X=data.drop(["label"],axis=1)
y=data["label"]

viewing img from dataset with index

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
index=34
image=X.loc[index].values.reshape(28,28)
plt.imshow(img)

In [ ]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.33)

In [ ]:
import joblib
from sklearn.svm import SVC
classifier=SVC(kernel='linear',random_state=2)
classifier.fit(train_X,train_y)
joblib.dump(classifier,'model/digit_recognizer')


In [ ]:
from sklearn import metrics
pred=classifier.predict(test_X)
metrics.accuracy_score(pred,test_y)

# Live prediction

In [ ]:
import pyscreenshot as ImageGrab

In [ ]:
model=joblib.load('model/digit_recognizer')
img_folder='img'
while True:
    img=ImageGrab.grab(bbox=(25,231,305,511))
    img.save(imgdataset+"img.png")
    im2=cv2.imread(imgdataset+"img.png")
    im_gray = cv2.cvtColor(im2,cv2.COLOR_BGR2GRAY)
    im_gray  =cv2.GaussianBlur(im_gray, (15,15), 0)
    
    #Threshold the image
    ret, im_th = cv2.threshold(im_gray,100, 255, cv2.THRESH_BINARY)
    roi = cv2.resize(im_th, (28,28), interpolation  =cv2.INTER_AREA)
    
    rows,cols=roi.shape
    
    X = []
    
    ## Add pixel one by one into data array
    for i in range(rows):
        for j in range(cols):
            k = roi[i,j]
            if k>100:
                k=1
            else:
                k=0
            X.append(k)
            
    predictions  =model.predict([X])
    print("Prediction:",predictions[0])
    cv2.putText(im2, "Prediction is: "+str(predictions[0]), (20,20), 0, 0.8,(0,255,0),2,cv2.LINE_AA)
    
    cv2.startWindowThread()
    cv2.namedWindow("Result")
    cv2.imshow("Result",im2)
    cv2.waitKey(10000)
    if cv2.waitKey(1)==13: #27 is the ascii value of esc, 13 is the ascii value of enter
        break
    cv2.destroyAllWindows()
 